In [67]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from IPython.display import display


pio.templates.default = "seaborn"
pd.options.plotting.backend = "plotly"

%matplotlib inline
%run global.ipynb

%store -r estimates_df

In [68]:
df = estimates_df
df

,date,steps,userId,gender,ageRange,created,stepsEstimate
0,2017-09-18,2421.0,5f772fc063993d0008607bb4,male,25-34,2020-10-02 13:48:48.153,0.25
1,2017-09-19,5718.0,5f772fc063993d0008607bb4,male,25-34,2020-10-02 13:48:48.153,0.25
2,2017-09-20,4542.0,5f772fc063993d0008607bb4,male,25-34,2020-10-02 13:48:48.153,0.25
3,2017-09-21,6021.0,5f772fc063993d0008607bb4,male,25-34,2020-10-02 13:48:48.153,0.25
4,2017-09-22,2970.0,5f772fc063993d0008607bb4,male,25-34,2020-10-02 13:48:48.153,0.25
...,...,...,...,...,...,...,...
1361803,2023-05-22,165.0,64784cba2562860008d7f574,female,45-54,2023-06-01 07:46:02.263,0.00
1361804,2023-05-25,18.0,64784cba2562860008d7f574,female,45-54,2023-06-01 07:46:02.263,0.00
1361805,2023-05-26,4687.0,64784cba2562860008d7f574,female,45-54,2023-06-01 07:46:02.263,0.00
1361806,2023-05-29,2481.0,64784cba2562860008d7f574,female,45-54,2023-06-01 07:46:02.263,0.00


In [69]:
# The specific date to split the data

num_months_before = 12
num_months_after = 8

split_date = pd.to_datetime('2020-03-16')
# split date minus 3 months
earliest_date = split_date - pd.DateOffset(months=num_months_before)
to_date = split_date + pd.DateOffset(months=num_months_after)
# days between the earliest date and the split date
days_in_before_period = (split_date - earliest_date).days
ratio_minimum_data = 1

fixed_outlier_steps_amount = 50000

In [70]:
# Assuming you have the necessary variables like `earliest_date`, `split_date`, and `ratio_minimum_data`

# Ensure all datetime columns are consistent (timezone-naive)
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
df['created'] = pd.to_datetime(df['created']).dt.tz_localize(None)

# Pre-filter the DataFrame
df_filtered = df[(df['date'] >= earliest_date)]

def process_user_data(user_data):
    user_id = user_data['userId'].iloc[0]
    created_date = user_data['created'].iloc[0]
    # created_date = to_date
    stepsEstimate = user_data['stepsEstimate'].iloc[0]
    gender = user_data['gender'].iloc[0]
    age_range = user_data['ageRange'].iloc[0]
    
    user_data_before = user_data[user_data['date'] < split_date]
    user_data_after = user_data[(user_data['date'] >= split_date) & (user_data['date'] < created_date)]
    
    days_in_after_period = (created_date - split_date).days
    
    steps_before = user_data_before['steps'].mean()
    steps_after = user_data_after['steps'].mean()

    # Calculate average steps for each month before and after split date
    steps_per_month_before = user_data_before.groupby(user_data_before['date'].dt.to_period('M'))['steps'].mean()
    steps_per_month_after = user_data_after.groupby(user_data_after['date'].dt.to_period('M'))['steps'].mean()
    
    if len(user_data_before) < days_in_before_period * ratio_minimum_data or len(user_data_after) < days_in_after_period * ratio_minimum_data:
        return None
    if gender == 'prefer not to say' or gender == 'other' or age_range == 'prefer not to say':
        return None
    if steps_before > fixed_outlier_steps_amount or steps_after > fixed_outlier_steps_amount:
        return None

    change = (steps_after - steps_before) / steps_before

    steps_per_month_before = steps_per_month_before.tail(num_months_before)
    steps_per_month_after = steps_per_month_after.head(num_months_after)
    
    result ={
        'userId': user_id,
        'stepsBefore': steps_before,
        'stepsAfter': steps_after,
        'daysBefore': len(user_data_before),
        'daysAfter': len(user_data_after),
        'stepsEstimate': stepsEstimate,
        'change': change,
        'gender': gender,
        'ageRange': age_range,
    }

    for i, (month, avg_steps) in enumerate(steps_per_month_before.items(), start=1):
        result[f'stepsMonthBefore_{i}'] = avg_steps

    for i, (month, avg_steps) in enumerate(steps_per_month_after.items(), start=1):
        result[f'stepsMonthAfter_{i}'] = avg_steps
    
    return pd.DataFrame([result])

# Group the data by userId and apply the function
all_users_df = df_filtered.groupby('userId').apply(process_user_data).dropna()

# Reset the index to get a clean DataFrame
all_users_df.reset_index(drop=True, inplace=True)

# Output the final DataFrame
all_users_df

,userId,stepsBefore,stepsAfter,daysBefore,daysAfter,stepsEstimate,change,gender,ageRange,stepsMonthBefore_1,...,stepsMonthBefore_11,stepsMonthBefore_12,stepsMonthAfter_1,stepsMonthAfter_2,stepsMonthAfter_3,stepsMonthAfter_4,stepsMonthAfter_5,stepsMonthAfter_6,stepsMonthAfter_7,stepsMonthAfter_8
0,5f8925fed211360008c5b33f,8061.450820,16212.748837,366,215,0.95,1.011145,male,45-54,8712.166667,...,6949.896552,7425.600000,8832.9375,15801.900000,17380.483871,14358.366667,13637.161290,18725.193548,20990.600000,16741.250000
1,5f892b9b6e872e0008c2bb3c,21172.456284,20606.525581,366,215,-0.45,-0.026730,male,35-44,23068.633333,...,18629.896552,19453.200000,21131.3125,20586.066667,23021.161290,22151.066667,19833.903226,21024.451613,18710.966667,16787.125000
2,5f892bc46e872e0008c56ea5,8100.404372,11477.386047,366,215,-0.25,0.416891,male,45-54,8245.633333,...,10280.620690,8709.200000,8592.0000,10611.533333,8978.774194,10000.833333,11312.838710,16048.935484,13291.866667,11655.125000
3,5f8953f442c60d0008c35575,8721.650273,10139.320930,366,215,-0.30,0.162546,female,25-34,9101.800000,...,9103.344828,7644.466667,9375.7500,11183.866667,10529.483871,9573.300000,8938.806452,10674.161290,10052.700000,10701.875000
4,5f8954c142c60d0008c55177,12600.166667,11953.316279,366,215,-0.15,-0.051337,male,35-44,13474.466667,...,12955.965517,15724.266667,12286.7500,14042.833333,12609.322581,11201.933333,12276.903226,12560.225806,10151.600000,9415.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,6048eacae581bc0008a6675f,12454.046448,11672.566667,366,360,-0.35,-0.062749,female,25-34,13458.201499,...,14421.758621,12637.933333,11301.1250,17298.233333,22044.523260,13699.259299,9779.419355,14324.451613,8093.866667,8923.645161
712,604a52f0d7d1c800084a3c82,5469.480874,5273.778393,366,361,-0.50,-0.035781,female,55-64,8021.400000,...,4848.000000,6057.600000,6612.1250,6831.833333,5020.677419,4493.200000,5199.869670,6101.581943,5796.366667,4282.161290
713,604a55b002123b0008467858,6913.898907,6152.331642,366,361,0.00,-0.110150,female,35-44,10171.800000,...,6350.482759,8661.933333,3732.3750,4543.233333,3678.258065,5077.100000,4983.935484,7664.387097,4962.300000,6150.539588
714,604c26316197960008f696fd,7615.696721,8688.181818,366,363,-0.20,0.140826,female,18-24,7667.543360,...,12554.689655,13657.733333,10677.2500,11655.766667,12182.747631,10636.220797,7916.006437,8515.677419,6426.200000,6070.580645


In [74]:

def plot_filtered_data(gender, age_range, all_users_df, split_date):

    filtered_df = all_users_df.copy()
    
    if gender != 'All':
        filtered_df = filtered_df[filtered_df['gender'] == gender]
        
    if age_range != 'All':
        filtered_df = filtered_df[filtered_df['ageRange'] == age_range]

    # Extract columns related to steps before and after
    months_before = [f'stepsMonthBefore_{i}' for i in range(1, num_months_before + 1)]
    months_after = [f'stepsMonthAfter_{i}' for i in range(1, num_months_after + 1)]
    
    # Calculate the mean steps across all users for each month
    avg_steps_before = filtered_df[months_before].mean()
    avg_steps_after = filtered_df[months_after].mean()
    
    # Combine before and after data for plotting
    avg_steps_combined = pd.concat([avg_steps_before, avg_steps_after], axis=0).reset_index(drop=True)
    
    # Calculate overall average before and after the split
    overall_avg_before = avg_steps_before.mean()
    overall_avg_after = avg_steps_after.mean()
    
    # Calculate the new line as averageBefore reduced by the averageEstimate percentage
    avg_estimate = filtered_df['stepsEstimate'].mean()
    avg_before_adjusted = overall_avg_before * (1 + avg_estimate)  # Subtract percentage (if avg_estimate is negative, this subtracts)

    # Create labels for x-axis with actual calendar months
    split_date = pd.to_datetime(split_date)
    months_before_labels = [(split_date - pd.DateOffset(months=num_months_before - i)).strftime('%b %Y') for i in range(num_months_before)]
    months_after_labels = [(split_date + pd.DateOffset(months=i)).strftime('%b %Y') for i in range(num_months_after)]
    labels = months_before_labels + months_after_labels
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(labels, avg_steps_combined, marker='o', linestyle='-', color='b')
    
    # Add a vertical line at the split point (between the last month before and the first month after)
    split_index = num_months_before  # This is where the split occurs (end of before, start of after)
    plt.axvline(x=split_index - 0.5, color='red', linestyle='--', label='Split Date')
    
    # Add horizontal lines for overall average steps before and after the split, stopping and starting at the split date
    plt.hlines(y=overall_avg_before, xmin=0, xmax=split_index - 1, color='green', linestyle='--', label='Average Before')
    plt.hlines(y=overall_avg_after, xmin=split_index, xmax=len(labels) - 1, color='purple', linestyle='--', label='Average After')
    
    # Add the new line in the after period: averageBefore adjusted by stepsEstimate percentage
    plt.hlines(y=avg_before_adjusted, xmin=split_index, xmax=len(labels) - 1, color='orange', linestyle='--', label='Adjusted Line (Before Adjusted)')
    
    plt.title(f'Average Steps Per Month\nGender: {gender}, Age Range: {age_range}')
    plt.xlabel('Month')
    plt.ylabel('Average Steps')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.ylim(0, max(avg_steps_combined.max(), overall_avg_before, overall_avg_after, avg_before_adjusted) * 1.1)

    plt.show()

# Define interactive widgets for gender and age range selection
gender_widget = widgets.Dropdown(
    options=['All'] + all_users_df['gender'].unique().tolist(),
    value='All',
    description='Gender:',
)

age_range_widget = widgets.Dropdown(
    options=['All'] + all_users_df['ageRange'].unique().tolist(),
    value='All',
    description='Age Range:',
)

# Function to update the plot based on widget changes
def update_plot(gender, age_range):
    plot_filtered_data(gender, age_range, all_users_df, split_date)

# Link the widgets to the update_plot function
ui = widgets.VBox([gender_widget, age_range_widget])
out = widgets.interactive_output(update_plot, {'gender': gender_widget, 'age_range': age_range_widget})

# Display the widgets and the plot
display(ui, out)

Output()

In [72]:
avg_estimate = all_users_df['stepsEstimate'].mean()

avg_estimate

-0.1337290502793296